In [1]:
import os
import cv2 
import pickle
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import plot_model
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten
from sklearn.metrics import confusion_matrix,classification_report,precision_score,recall_score,f1_score

C:\Users\Lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
project_dir = '/content/drive/My Drive/Project_int428'
categories  = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
data = []

In [3]:
for category in categories:
  path = os.path.join(project_dir+'/flowers', category)
  label = categories.index(category)
  for img_name in tqdm(os.listdir(path)):
    image_path = os.path.join(path,img_name)
    try:
      image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
      image = cv2.resize(image, (224,224))
      for i in range(0,3):
        data.append([image,label])
    except Exception as e:
      pass

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/Project_int428/flowers\\daisy'

In [ ]:
np.random.shuffle(data)

In [ ]:
feature = []
labels = []
for img, label in data:
    feature.append(img)
    labels.append(label)

In [ ]:
del data[:]
del data

In [ ]:
feature = np.array(feature, dtype=np.float32)
labels = np.array(labels)
feature = feature/255.0
feature = np.array(feature).reshape(-1, 224, 224, 1)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(feature,labels,test_size=0.3)

In [ ]:
del feature
del labels

In [ ]:
X_train.shape

In [ ]:
lyr_size = 64
conv_lyr = 3

In [ ]:
model = Sequential()

model.add(Conv2D(lyr_size, (3, 3), input_shape=X_train.shape[1:],name='Conv2D_layer_1'))
model.add(Activation('relu',name='Activation_layer_1'))
model.add(MaxPooling2D(pool_size=(2, 2),name='MaxPooling2D_layer_1'))

for l in range(conv_lyr):
    model.add(Conv2D(lyr_size, (3, 3),name='Conv2D_layer_'+str(l+2)))
    model.add(Activation('relu',name='Activation_layer_'+str(l+2)))
    model.add(MaxPooling2D(pool_size=(2, 2),name='MaxPooling2D_layer_'+str(l+2)))

model.add(Flatten(name='Flatten_layer'))
model.add(Dense(64,name='Dense_layer_1'))
model.add(Activation('relu',name='Activation_layer_5'))
model.add(Dense(5,name='Dense_layer_2'))
model.add(Activation('softmax',name='Activation_layer_6'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(X_train,y_train,batch_size=250,epochs=10)

In [ ]:
del X_train
del y_train

In [ ]:
model.save(project_dir+'/flower_model.h5')

In [ ]:
evaluation=model.evaluate(X_test,y_test)

In [ ]:
plot_model(model, to_file=project_dir+'/Model_Diagram.png', show_shapes=True, show_layer_names=True)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('Model Training')
plt.xlabel('Epoch')
plt.legend(['Train Accuracy', 'Train Loss'], loc='upper right')
plt.savefig(project_dir+'/Model_Training.png')
plt.show()

In [ ]:
pred=model.predict(X_test)
y_pred=[]
for i in pred:
  y_pred.append(np.argmax(i))

In [ ]:
print(classification_report(y_test, y_pred,target_names=categories))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
group_percentages = []
for i in range(0,len(cm)):
  group_percentages.append(np.round(cm[i]/np.sum(cm[i]),2))
df_cm = pd.DataFrame(group_percentages, categories, categories)
plt.figure(figsize=(15,10))
sn.heatmap(df_cm, annot=True, annot_kws={"size": 12},cmap='Blues')
plt.title("Confusion_Matrix")
plt.savefig(project_dir+'/Confusion_Matrix.png')
plt.show()
plt.close()

In [ ]:
properties=["Accuracy","F1 Score","Precision","Recall"]

df=pd.DataFrame()

l=[]
l.append(round(evaluation[1],3))
l.append(round(f1_score(y_test,y_pred, average='weighted'),3))
l.append(round(precision_score(y_test,y_pred, average='weighted'),3))
l.append(round(recall_score(y_test,y_pred, average='weighted'),3))

df['Property']=properties
df['Value']=l
print(df)

In [ ]:
sn.set(rc={'figure.figsize':(8,5)},style="whitegrid")
a=sn.barplot(y="Property",x="Value",data=df)
for p in a.patches:
  width = p.get_width()
  a.text(width-0.09,p.get_y()+p.get_height()/2,("{:.2f}".format(width)),ha = 'left',va = 'center')
plt.title("Model Scores")
plt.savefig(project_dir+'/Model_Scores.png')
plt.show()
plt.close()
del a

In [ ]:
plt.figure(figsize=(13,12)).suptitle('Model Validation', fontsize=15)
for i in range(0,9):
  plt.subplot(3,3,i+1)
  plt.imshow(np.array(X_test[i]).reshape(224, 224),cmap='gray')
  plt.xticks([])
  plt.yticks([])
  plt.xlabel('Actual:'+categories[y_test[i]]+'\nPredicate:'+categories[np.argmax(pred[i])])
plt.savefig(project_dir+'/Model_Validation.png')
plt.show()
plt.close()